In [72]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
from collections import defaultdict

In [20]:
user_agent = UserAgent()
options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--disable-notifications')
options.add_argument('--disable-popup-blocking')
options.add_argument(f'user-agent={user_agent.random}')
# options.add_argument('--incognito')

In [70]:
base_url = 'https://www.mobbi.id'
url = 'https://www.mobbi.id/cari-mobil?q=%3Alatest-listing%3Abrand%3ATOYOTA%3AproductStatus%3AApa%2BAdanya%3AproductStatus%3ASudah%2BDiinspeksi%3AproductStatus%3ASiap%2BPakai'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

cars = soup.find_all('div', class_="col-6 col-sm-6 col-md-4 col-lg-3 col-xl-3 px-2")

# Create lists to store the data
product_links = []
product_brands = []
product_models = []
product_variants = []
product_prices = []
product_transmissions = []
product_mileages = []
product_years = []
product_locations = []

In [85]:
base_url = 'https://www.mobbi.id'
url = 'https://www.mobbi.id/cari-mobil?q=%3Alatest-listing%3Abrand%3ATOYOTA%3AproductStatus%3AApa%2BAdanya%3AproductStatus%3ASudah%2BDiinspeksi%3AproductStatus%3ASiap%2BPakai'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

cars = soup.find_all('div', class_='col-6 col-sm-6 col-md-4 col-lg-3 col-xl-3 px-2')

# Create a list to store the data
data_list = []

for car in cars:
    product_name = car.find('h5', class_='product-name')
    if product_name is not None:
        element = car.find('div', class_='featured-car-product for-compare-button no-rounded-bottom')
        data = {}
        data['Product Link'] = base_url + car.div.a['href'] if car.div.a else None
        if element:
            data['Product Brand'] = element.get('data-product-brand')
            data['Product Model'] = element.get('data-product-category')
            data['Product Variant'] = element.get('data-product-variant')
            data['Product Price'] = element.get('data-product-price')
            data['Product Transmission'] = element.get('data-product-transmission')
            data['Product Mileage'] = element.get('data-product-mileage')
            data['Product Year'] = element.get('data-product-year')
            data['Product Location'] = element.get('data-product-location')
        else:
            # If element is not found, set the corresponding values to None 
            data.update({
                'Product Brand': None,
                'Product Model': None,
                'Product Variant': None,
                'Product Price': None,
                'Product Transmission': None,
                'Product Mileage': None,
                'Product Year': None,
                'Product Location': None
            })
        data_list.append(data)

# Create a Pandas DataFrame
df = pd.DataFrame(data_list)
df


,Product Link,Product Brand,Product Model,Product Variant,Product Price,Product Transmission,Product Mileage,Product Year,Product Location
0,https://www.mobbi.id/in/mobil-bekas/toyota-cx5...,Toyota,CX5,CX5 GT,5.1E+8,AT,24506,2020,Kota Tangerang Selatan
1,https://www.mobbi.id/in/mobil-bekas/toyota-for...,Toyota,FORTUNER,FORTUNER 2.4 VRZ TRD AT DIESEL (NEW),5.253E+8,AT,53103,2020,Kota Tangerang Selatan
2,https://www.mobbi.id/in/mobil-bekas/toyota-agy...,Toyota,AGYA,AGYA 1.2 G AT TRD BENSIN (ALL NEW),1.683E+8,AT,14002,2021,Kota Bekasi
3,https://www.mobbi.id/in/mobil-bekas/toyota-ava...,Toyota,AVANZA,VELOZ 1.5 Q CVT,2.856E+8,AT,15338,2022,Kota Bekasi
4,https://www.mobbi.id/in/mobil-bekas/toyota-rai...,Toyota,RAIZE,RAIZE 1.0T GR SPORT CVT TSS,2.805E+8,AT,15244,2021,Kota Bekasi
5,https://www.mobbi.id/in/mobil-bekas/toyota-yar...,Toyota,YARIS,YARIS 1.5 S CVT GR SPORT,2.856E+8,AT,10634,2021,Kota Bekasi
6,https://www.mobbi.id/in/mobil-bekas/toyota-inn...,Toyota,INNOVA,V (INNOVA),4.233E+8,AT,23178,2022,Kota Tangerang Selatan
7,https://www.mobbi.id/in/mobil-bekas/toyota-rai...,Toyota,RAIZE,GR TURBO SPORT TSS 2T (RAIZE),2.6E+8,AT,11878,2022,Kota Bandung
8,https://www.mobbi.id/in/mobil-bekas/toyota-inn...,Toyota,INNOVA,2.0 V,4.131E+8,AT,17053,2022,Kota Tangerang Selatan
9,https://www.mobbi.id/in/mobil-bekas/toyota-for...,Toyota,FORTUNER,FORTUNER 2.4 VRZ AT DIESEL (NEW),4.692E+8,AT,86470,2019,Kota Tangerang Selatan
